# Step by Step variational classification with Perceval

In [ ]:
import perceval as pcvl
import numpy as np
from scipy.optimize import minimize
import time
import tqdm

## Data loading and normalization

In [ ]:
# See qiskit notebook
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

iris_data = load_iris()
features = iris_data.data
labels = iris_data.target

In [ ]:
# normalize the features
features = MinMaxScaler().fit_transform(features)
# split train, test
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.8, random_state=123
)

## A first draft
We want a circuit composed of 3 blocks - see https://opg.optica.org/abstract.cfm?uri=CLEO_AT-2021-JW1A.73

In [ ]:
m = 4
parameters = np.random.normal(size=4*m**2)

px1, px2, px3, px4 = pcvl.P("px1"),pcvl.P("px2"),pcvl.P("px3"),pcvl.P("px4")
c = pcvl.Unitary(pcvl.Matrix.parametrized_unitary(m, parameters[:2 * m ** 2]), name="W1")\
     // (0, pcvl.PS(px1)) // (1, pcvl.PS(px2)) // (2, pcvl.PS(px3)) // (3, pcvl.PS(px4))\
     // pcvl.Unitary(pcvl.Matrix.parametrized_unitary(m, parameters[2 * m ** 2:]), name="W2")

backend = pcvl.BackendFactory().get_backend("SLOS")
backend.set_circuit(pcvl.Unitary(pcvl.Matrix.random_unitary(m)))
backend.preprocess([pcvl.BasicState([1]*m)])

pcvl.pdisplay(c)

However, we won't do such a dynamic circuit since compilation will shuffle all the parameters around when the unitary will be built - let us directly control the phases on our QPU

## Physical Implementation on QPU Ascella

First let us get a token on https://cloud.quandela.com

In [ ]:
qpu_token = "YOUR TOKEN"
processor = pcvl.RemoteProcessor("qpu:ascella", qpu_token)
circuit = processor.specs["specific_circuit"]
circuit_phases = circuit.get_parameters()

In [ ]:
# We have retrieved the circuit corresponding to Ascella
pcvl.pdisplay(circuit)

### Chip manual identification of phases and selection of phases

In [ ]:
# set all phases to 0 => no energy used
for p in circuit_phases:
    p.set_value(0)
    
# let us change phases after the 4 first modes to build a reflexive boundary
boundary = [14,36,58,80,102,122]
for b in boundary:
    circuit_phases[b].set_value(np.pi)

# let us select parameters for weights W1/W2, and parameters (we need 4)
weight_left = [0, 1, 2, 3] # we could add these ones , 22, 23, 24, 25]
circuit_params = [44, 46, 66, 68 ]
weight_right = [88, 89, 90, 91] # we could add these ones , 110, 111, 112, 113]
n_weights= len(weight_left)+len(weight_right)

# visualize the weights and params by setting resp value 1, 2, 3
for w in weight_left:
    circuit_phases[w].set_value(1)
for p in params:
    circuit_phases[p].set_value(2)
for w in weight_right:
    circuit_phases[w].set_value(3)

pcvl.pdisplay(circuit)

### Configuration of the processor

In [ ]:
# We are using here a simulator, but we could reuse the processor above for run on QPU
sim_processor = pcvl.Processor("SLOS", circuit)
sim_processor.with_input(pcvl.BasicState([0,1,0,1]+[0]*8))

### Utility functions - computation of the result, and loss

In [ ]:
def computation(w, params):
    if w is not None:
        # if we don't provide weights, we keep last values
        for idx, w in enumerate(w):
            if idx < len(weight_left):
                circuit_phases[weight_left[idx]].set_value(w*2*np.pi)
            else:
                circuit_phases[weight_right[idx-len(weight_left)]].set_value(w*2*np.pi)

    for idx, p in enumerate(params):
        circuit_phases[circuit_params[idx]].set_value(p*2*np.pi)

    # We sample from the processor defined above
    sampler = pcvl.algorithm.Sampler(sim_processor)
    results = sampler.sample_count(1024)
    
    return results

In [ ]:
# let us test how this works with random weights and parameters
r = computation([0.2]*n_weights,[0]*4)

In [ ]:
pcvl.pdisplay(r["results"])

In [ ]:
# we map the states without bunching to the 3 features
# 1001... and 0110... => class 1
# 0011... and 1100... => class 2
# 1010... and 0101... => class 3
# otherwise ignored

def map(output_state):
    so=str(output_state[0:4])
    if so=="|1,0,0,1>" or so=="|0,1,1,0>":
        return 1
    if so=="|0,0,1,1>" or so=="|1,1,0,0>":
        return 2
    if so=="|1,0,1,0>" or so=="|0,1,0,1>":
        return 3
    return 0

In [ ]:
# test mapping
map(pcvl.BasicState("|1,1,0,0,0,0,0,0,0,0,0,0>"))

In [ ]:
# calculate the loss function as the distance to the class probability

def loss(w, params, label):
    o = computation(w, params)["results"]
    total_counts = 0
    counts = [0, 0, 0]
    for bs, count in o.items():
        the_class = map(bs)
        if the_class:
            counts[the_class-1] += count
            total_counts += count
    return 1-counts[label-1]/total_counts

In [ ]:
# and select the "best" label (the one with highest probability)

def best_label(w, params):
    o = computation(w, params)["results"]
    counts = [0, 0, 0]
    for bs, count in o.items():
        the_class = map(bs)
        if the_class:
            counts[the_class-1] += count
    max_count = max(counts)
    return counts.index(max_count)+1

In [ ]:
# test loss function and best_label selection
print(loss([0.2]*n_weights,[0.1]*4, 3))
print(best_label(None,[0.1]*4))

In [ ]:
# total loss for all the train sample
current_loss = 0
total_computation = 0
def total_loss(w):
    global current_loss
    global total_computation
    total_loss = 0
    set_w = w
    for feat, label in zip(train_features, train_labels):
        total_loss += loss(set_w, list(feat), label+1)
        set_w = None
    current_loss = total_loss/len(train_features)
    total_computation += 1
    return current_loss

In [ ]:
# accuracy of the classification for set of features/labels
def accuracy(features, labels):
    acc = 0
    for feat, label in zip(features, labels):
        pred_label = best_label(None, list(feat))
        if pred_label == label+1:
            acc += 1
    return acc/len(features)

In [ ]:
total_loss([0.2]*n_weights)
accuracy(train_features, train_labels)

In [ ]:
# callback function that will allow us to see how the run progress
def callbackF(parameters):
    """callback function called by scipy.optimize.minimize allowing to monitor progress"""
    global current_loss
    global loss_evolution
    global total_computation
    global start_time
    now = time.time()
    pbar.set_description("Loss: %0.5f elapsed: %0.5f total computation: %d accuracy: %f" %
                         (current_loss, now-start_time, total_computation, accuracy(test_features, test_labels)))
    pbar.update(1)
    loss_evolution.append((current_loss, now-start_time))
    computation_count = 0
    start_time = now

### The classical optimization

In [ ]:
current_loss = 0
start_time = time.time()
loss_evolution = []

weights = np.random.normal(size=n_weights)

pbar = tqdm.tqdm()
res = minimize(total_loss, weights, callback=callbackF, method='Nelder-Mead')

### What's next?

With the 8 parameters, we reach a plateau around 50% accuracy - we can do better though:

* Increase the number of parameters
* Smarter mapping function - it can even be learnt
* Smarter Ansatz - not just random
* Data re-uploading

See perceval_tutorial_variational_classifier for a full implementation with almost 100% accuracy!